Importing Modules and Loading Data

In [ ]:
import numpy as np
import pandas as pd
import os
import csv
import gensim
import re
import matplotlib.pyplot as plt
import warnings


from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import string
data = pd.read_csv('/content/drive/MyDrive/Major/twitter_data.csv',header=0)
data['User'] = data['User'].astype('category')
label_mapping = data['User'].cat.categories 
data['User'] = data['User'].cat.codes
X = data.drop('User', axis = 1)
y = data.User

Data Pre-Processing

In [4]:
#Converting to Lowercase
X['clean_text'] = X['Tweet'].str.lower()
X.head()

,Tweet,clean_text
0,Tune in 11:30 ET tomorrow for a live webcast o...,tune in 11:30 et tomorrow for a live webcast o...
1,"Iowans, there's a good chance there's a Biden ...","iowans, there's a good chance there's a biden ..."
2,We're excited to announce that @JoeBiden is be...,we're excited to announce that @joebiden is be...
3,Campaign staff will run this account to keep y...,campaign staff will run this account to keep y...
4,News for you this morning: VP Biden will speak...,news for you this morning: vp biden will speak...


In [5]:
#Removing Url's from Tweets
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    text = url_pattern.sub('', text)
    return text

In [6]:
X['clean_text'] = X['clean_text'].apply(lambda x: remove_urls(x))
X.head()

,Tweet,clean_text
0,Tune in 11:30 ET tomorrow for a live webcast o...,tune in 11:30 et tomorrow for a live webcast o...
1,"Iowans, there's a good chance there's a Biden ...","iowans, there's a good chance there's a biden ..."
2,We're excited to announce that @JoeBiden is be...,we're excited to announce that @joebiden is be...
3,Campaign staff will run this account to keep y...,campaign staff will run this account to keep y...
4,News for you this morning: VP Biden will speak...,news for you this morning: vp biden will speak...


In [7]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
#Removing Punctuations from Tweets
def remove_punctuations(text):
    punctuations = string.punctuation
    return text.translate(str.maketrans('', '', punctuations))

In [9]:
X['clean_text'] = X['clean_text'].apply(lambda x: remove_punctuations(x))
print(X['clean_text'])

0        tune in 1130 et tomorrow for a live webcast of...
1        iowans theres a good chance theres a biden nea...
2        were excited to announce that joebiden is bein...
3        campaign staff will run this account to keep y...
4        news for you this morning vp biden will speak ...
                               ...                        
88205    25 reasons that 25 people are voting for presi...
88206    at the final rally of his final campaign last ...
88207    it’s election day this is your last chance to ...
88208    election day is here confirm your polling plac...
88209    president obama tells the story of fired up re...
Name: clean_text, Length: 88210, dtype: object


In [10]:
#importing stopwords
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [11]:
#Removal of Stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in text.split() if word not in STOPWORDS])

In [12]:
X['clean_text'] = X['clean_text'].apply(lambda x: remove_stopwords(x))
X.head()

,Tweet,clean_text
0,Tune in 11:30 ET tomorrow for a live webcast o...,tune 1130 et tomorrow live webcast families us...
1,"Iowans, there's a good chance there's a Biden ...",iowans theres good chance theres biden near to...
2,We're excited to announce that @JoeBiden is be...,excited announce joebiden rebooted 2012 campai...
3,Campaign staff will run this account to keep y...,campaign staff run account keep date vps youll...
4,News for you this morning: VP Biden will speak...,news morning vp biden speak exeter nh 412 tax ...


In [13]:
# Finding Common words and their Corresponding Frequencies
from collections import Counter
word_count = Counter()
for text in X['clean_text']:
    for word in text.split():
        word_count[text] += 1
        
word_count.most_common(10)

[('rt realdonaldtrump', 268),
 ('make america great', 234),
 ('rt whitehouse live press briefing coronavirus task force', 232),
 ('rt whitehouse live president realdonaldtrump holds news conference', 224),
 ('days congress might allow series arbitrary budget cuts take place—known washington sequester” jobsovercuts 201302261023 live president obama speaks virginia congress prevent severe spending cuts jobsovercuts 201302260916 voice heard share cant afford lay teachers cut vital services kids jobsovercuts 201302260830 read rt president obamas plan prevent severe budget cuts vs republican inaction jobsovercuts 201302251643 focus creating jobs cutting vital services middleclass families jobsnotcutspictwittercom8aijglkksy 201302251423 family cant afford severe spending cuts vital services kids seniors jobsnotcuts 201302251156 agree time republicans congress protect middleclass families—not tax loopholes millionaires jobsnotcuts 201302251130 congress refuses act friday 370000 mentally ill c

In [14]:
#Removal of Frequent Words
FREQUENT_WORDS = set(word for (word, wc) in word_count.most_common(3))
def remove_freq_words(text):
    return " ".join([word for word in text.split() if word not in FREQUENT_WORDS])

In [15]:
X['clean_text'] = X['clean_text'].apply(lambda x: remove_freq_words(x))
X.head()

,Tweet,clean_text
0,Tune in 11:30 ET tomorrow for a live webcast o...,tune 1130 et tomorrow live webcast families us...
1,"Iowans, there's a good chance there's a Biden ...",iowans theres good chance theres biden near to...
2,We're excited to announce that @JoeBiden is be...,excited announce joebiden rebooted 2012 campai...
3,Campaign staff will run this account to keep y...,campaign staff run account keep date vps youll...
4,News for you this morning: VP Biden will speak...,news morning vp biden speak exeter nh 412 tax ...


In [16]:
tfidf_data = X['clean_text']
print(tfidf_data)

0        tune 1130 et tomorrow live webcast families us...
1        iowans theres good chance theres biden near to...
2        excited announce joebiden rebooted 2012 campai...
3        campaign staff run account keep date vps youll...
4        news morning vp biden speak exeter nh 412 tax ...
                               ...                        
88205          25 reasons 25 people voting president obama
88206    final rally final campaign last night presiden...
88207    it’s election day last chance help win thing—r...
88208    election day confirm polling place bring frien...
88209    president obama tells story fired ready go fin...
Name: clean_text, Length: 88210, dtype: object


In [17]:
#Converting Series to array
a = np.array(tfidf_data.values.tolist())
print(a)

['tune 1130 et tomorrow live webcast families usa presidential forum health care'
 'iowans theres good chance theres biden near today cool 14 f day'
 'excited announce joebiden rebooted 2012 campaign season give news vice president trail'
 ...
 'it’s election day last chance help win thing—rt link friends know vote'
 'election day confirm polling place bring friend vote pictwittercomqq2nqhfd'
 'president obama tells story fired ready go final rally final campaign forward']


In [18]:
# Tokenizing words 
import nltk
nltk.download('punkt')
X['clean_text'] = X['clean_text'].apply(lambda x: word_tokenize(x))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
X = X['clean_text']

In [20]:
#Training the Word2Vec Model
model = gensim.models.Word2Vec(X, vector_size=75, window=5, min_count=5, workers=4, sg=1)

In [21]:
#Save Word2vec Model
model.save('word2vec_major.model')

In [22]:
#Load Word2Vec Model
from gensim.models import Word2Vec
Word2vec_model = Word2Vec.load('word2vec_major.model')
word = "excited"
sim_words = Word2vec_model.wv.most_similar(word, topn=10)

In [23]:
print(word)
for w, s in sim_words:
    print(f'{w}: {s}')

excited
fundraiser: 0.8466081023216248
traveling: 0.8249748349189758
kick: 0.8166437745094299
vincemcmahon: 0.8146267533302307
inducted: 0.8138881325721741
mt: 0.8120653629302979
firefighter: 0.8088599443435669
rsvp: 0.8080118298530579
⁦realdonaldtrump⁩: 0.8079483509063721
celeb: 0.8072080016136169


In [24]:
print(X)

0        [tune, 1130, et, tomorrow, live, webcast, fami...
1        [iowans, theres, good, chance, theres, biden, ...
2        [excited, announce, joebiden, rebooted, 2012, ...
3        [campaign, staff, run, account, keep, date, vp...
4        [news, morning, vp, biden, speak, exeter, nh, ...
                               ...                        
88205    [25, reasons, 25, people, voting, president, o...
88206    [final, rally, final, campaign, last, night, p...
88207    [it, ’, s, election, day, last, chance, help, ...
88208    [election, day, confirm, polling, place, bring...
88209    [president, obama, tells, story, fired, ready,...
Name: clean_text, Length: 88210, dtype: object


In [25]:
#Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(a, y, test_size=0.2, random_state=42)

Feature Extraction

In [26]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
#Extracting features using TF-IDF
tfidf = TfidfVectorizer()
tfidf_train_features = tfidf.fit_transform(X_train)
tfidf_test_features = tfidf.transform(X_test)
tfidf_train_features = tfidf_train_features.toarray()
tfidf_test_features = tfidf_test_features.toarray()

In [28]:
print(tfidf_train_features.shape)
print(y_train.shape)

(70568, 75096)
(70568,)


In [29]:
#filter out invalid features
train_valid_features = []
train_valid_features_names = []
for i in range(tfidf_train_features.shape[1]):
    feature_values = tfidf_train_features[:, i]
    if np.max(feature_values) > 0.8:
        train_valid_features.append(i)
        train_valid_features_names.append(tfidf.get_feature_names_out()[i])

In [30]:
# convert train and test features to Word2vec word vectors and pad sequences
train_word_vectors = []
test_word_vectors = []
max_seq_len = 20 
for i, word_vectors in [(tfidf_train_features, train_word_vectors), (tfidf_test_features, test_word_vectors)]:
    print(i.shape)
    sequences = []
    for k in range(i.shape[1]):
        if k not in train_valid_features:
            continue
        feature_name = train_valid_features_names[train_valid_features.index(k)]
        try:
            word_vector = Word2vec_model.wv[feature_name]
            sequences.append(word_vector)
        except KeyError:
            pass
    sequences = np.array(sequences)
    if sequences.shape[0] > max_seq_len: 
        sequences = sequences[:max_seq_len, :]
    word_vectors = np.zeros((i.shape[0], max_seq_len, Word2vec_model.vector_size))
    word_vectors[:, :sequences.shape[0], :] = sequences
    word_vectors = np.expand_dims(word_vectors, axis=-1)
    if i is tfidf_train_features:
        train_word_vectors.append(word_vectors)
    else:
        test_word_vectors.append(word_vectors)

train_word_vectors = np.concatenate(train_word_vectors, axis=0)
test_word_vectors = np.concatenate(test_word_vectors, axis=0)

(70568, 75096)
(17642, 75096)


In [31]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_labels = le.fit_transform(y_train)
test_labels =  le.transform(y_test)

In [32]:
num_classes = len(le.classes_)
train_one_hot_labels = np.zeros((len(train_labels), num_classes))
train_one_hot_labels[np.arange(len(train_labels)), train_labels] = 1
test_one_hot_labels = np.zeros((len(test_labels), num_classes))
test_one_hot_labels[np.arange(len(test_labels)), test_labels] = 1

Classification

In [33]:
#Create and Compile Model
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, Dense, Dropout, Flatten, GlobalMaxPool1D, Reshape
from keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(max_seq_len, Word2vec_model.vector_size)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
filepath = "weights_best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor="val_acc", verbose=1, save_best_only=True, mode='max', save_weights_only=True)
callbacks_list = [checkpoint]

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 18, 64)            14464     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 9, 64)            0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 7, 32)             6176      
                                                                 
 global_max_pooling1d (Globa  (None, 32)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 100)               3300      
                                                                 
 dropout (Dropout)           (None, 100)               0

In [34]:
#Fit And Model Evaluation
history = model.fit(train_word_vectors, train_one_hot_labels, epochs=20, batch_size=200, verbose=1, callbacks= callbacks_list, validation_data=(test_word_vectors, test_one_hot_labels))

Epoch 1/20
349/353 [============================>.] - ETA: 0s - loss: 1.0126 - accuracy: 0.6377

353/353 [==============================] - 5s 12ms/step - loss: 1.0123 - accuracy: 0.6379 - val_loss: 0.9926 - val_accuracy: 0.6441
Epoch 2/20
349/353 [============================>.] - ETA: 0s - loss: 1.0033 - accuracy: 0.6407

353/353 [==============================] - 5s 14ms/step - loss: 1.0037 - accuracy: 0.6406 - val_loss: 0.9921 - val_accuracy: 0.6441
Epoch 3/20
352/353 [============================>.] - ETA: 0s - loss: 1.0025 - accuracy: 0.6406

353/353 [==============================] - 4s 11ms/step - loss: 1.0025 - accuracy: 0.6406 - val_loss: 0.9910 - val_accuracy: 0.6441
Epoch 4/20
348/353 [============================>.] - ETA: 0s - loss: 1.0025 - accuracy: 0.6406

353/353 [==============================] - 4s 11ms/step - loss: 1.0023 - accuracy: 0.6406 - val_loss: 0.9901 - val_accuracy: 0.6441
Epoch 5/20
353/353 [==============================] - ETA: 0s - loss: 1.0017 - accuracy: 0.6406

...

353/353 [==============================] - 4s 11ms/step - loss: 0.9996 - accuracy: 0.6406 - val_loss: 0.9900 - val_accuracy: 0.6441
Epoch 17/20
350/353 [============================>.] - ETA: 0s - loss: 0.9997 - accuracy: 0.6405

353/353 [==============================] - 4s 12ms/step - loss: 0.9995 - accuracy: 0.6406 - val_loss: 0.9897 - val_accuracy: 0.6441
Epoch 18/20
348/353 [============================>.] - ETA: 0s - loss: 0.9989 - accuracy: 0.6406

353/353 [==============================] - 4s 12ms/step - loss: 0.9990 - accuracy: 0.6406 - val_loss: 0.9898 - val_accuracy: 0.6441
Epoch 19/20
348/353 [============================>.] - ETA: 0s - loss: 0.9992 - accuracy: 0.6406

353/353 [==============================] - 4s 11ms/step - loss: 0.9990 - accuracy: 0.6406 - val_loss: 0.9899 - val_accuracy: 0.6441
Epoch 20/20
351/353 [============================>.] - ETA: 0s - loss: 0.9990 - accuracy: 0.6406

353/353 [==============================] - 4s 12ms/step - loss: 0.9989 - accuracy: 0.6406 - val_loss: 0.9897 - val_accuracy: 0.6441


In [35]:
fig, ax1 = plt.subplots()

# Plotting the training and validation loss
ax1.plot(history.history['loss'], label='Training Loss')
ax1.plot(history.history['val_loss'], label='Validation Loss')
ax1.set_xlabel('Epoch (count)')
ax1.set_ylabel('Loss')
ax1.set_ylim([0.97,1.016])
ax1.legend(loc='upper left', bbox_to_anchor=(0., 0.2))

# Creating a twin y-axis for accuracy
ax2 = ax1.twinx()

# Plotting the training and validation accuracy
ax2.plot(history.history['accuracy'], label='Training Accuracy', color='green')
ax2.plot(history.history['val_accuracy'], label='Validation Accuracy', color='purple')
ax2.set_ylabel('Accuracy')
ax2.set_ylim([0.635,0.650])  # Set the range of the accuracy axis
ax2.legend(loc='upper right', bbox_to_anchor=(1., 0.2))

plt.title('Training and Validation Loss and Accuracy')
plt.show()


In [36]:
#Predicting With Test Data
from sklearn.metrics import classification_report

y_pred_test = model.predict(test_word_vectors)
y_pred_test_classes = np.argmax(y_pred_test, axis=1)
target_names = ['Joe Biden', 'Narendra Modi', 'Trump', 'Obama'] 


552/552 [==============================] - 1s 2ms/step


In [37]:
y_pred_train = model.predict(train_word_vectors)
y_pred_train_classes = np.argmax( y_pred_train, axis=1)

2206/2206 [==============================] - 3s 1ms/step


In [38]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [39]:
precision_score(y_test, y_pred_test_classes, average = 'micro' )

0.644144654801043

In [40]:
accuracy_score(y_test, y_pred_test_classes)

0.644144654801043

In [41]:
recall_score(y_test, y_pred_test_classes, average = 'micro' )

0.644144654801043

In [42]:
y.value_counts()

3    56571
1    18724
2     6851
0     6064
Name: User, dtype: int64